In [1]:
__authors__ = "Tianyuan Zhang"
__credits__ = ["Tianyuan Zhang", "Jeffrey B. Schriber", "Daniel G. A. Smith"]

__copyright__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__ = "BSD-3-Clause"
__date__ = "2017-05-26"

import time
import numpy as np
np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4

# Memory for Psi4 in GB
psi4.core.set_output_file('output.dat', False)

# Memory for numpy in GB
numpy_memory = 2

#mol = psi4.geometry("""
#O
#H 1 1.1
#H 1 1.1 2 104
#symmetry c1
#""")
mol = psi4.geometry("""
H
H 1 1.0
symmetry c1
""")

psi4.set_options({'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

print('\nStarting SCF and integral build...')





Starting SCF and integral build...


In [2]:
t = time.time()

# First compute SCF energy using Psi4
scf_e, wfn = psi4.energy('SCF', return_wfn=True)

# Grab data from wavfunction class
C = wfn.Ca()
ndocc = wfn.doccpi()[0]
nmo = wfn.nmo()
nvirt = nmo - ndocc
nDet_S = ndocc * nvirt * 2

# Compute size of SO-ERI tensor in GB
ERI_Size = (nmo**4) * 128e-9
print('\nSize of the SO ERI tensor will be %4.2f GB.' % ERI_Size)
memory_footprint = ERI_Size * 5.2
if memory_footprint > numpy_memory:
    clean()
    raise Exception("Estimated memory utilization (%4.2f GB) exceeds numpy_memory \
                    limit of %4.2f GB." % (memory_footprint, numpy_memory))

# Integral generation from Psi4's MintsHelper
t = time.time()
mints = psi4.core.MintsHelper(wfn.basisset())
H = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential())

print('\nTotal time taken for ERI integrals: %.3f seconds.\n' % (time.time() - t))
print(H)


Size of the SO ERI tensor will be 0.00 GB.

Total time taken for ERI integrals: 0.001 seconds.

[[-0.9795  -0.68286]
 [-0.68286 -0.9795 ]]


In [3]:
print('Starting AO -> spin-orbital MO transformation...')
t = time.time()
MO = np.asarray(mints.mo_spin_eri(C, C))
#print(MO)
# Update H, transform to MO basis and tile for alpha/beta spin
H = np.einsum('uj,vi,uv', C, C, H)
print('..finished transformation in %.3f seconds.\n' % (time.time() - t))
print(H)

Starting AO -> spin-orbital MO transformation...
..finished transformation in 0.001 seconds.

[[-1.11084  0.     ]
 [ 0.      -0.58912]]


In [4]:
H = np.repeat(H, 2, axis=0)
H = np.repeat(H, 2, axis=1)

# Make H block diagonal
spin_ind = np.arange(H.shape[0], dtype=np.int) % 2
H *= (spin_ind.reshape(-1, 1) == spin_ind)

print('..finished transformation in %.3f seconds.\n' % (time.time() - t))



..finished transformation in 24.635 seconds.



/var/folders/5f/ftf5xlgj4dv_6xxqh_p9pwyc0000gq/T/ipykernel_9462/2790351766.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  spin_ind = np.arange(H.shape[0], dtype=np.int) % 2


In [5]:
from helper_CI import Determinant, HamiltonianGenerator
from itertools import combinations

print('Generating %d CIS singlet Determinants...' % (nDet_S + 1))
t = time.time()

occList = [i for i in range(ndocc)]
det_ref = Determinant(alphaObtList=occList, betaObtList=occList)
detList = det_ref.generateSingleExcitationsOfDet(nmo)
detList.append(det_ref)
spin_obtlist = det_ref.getOrbitalMixedIndexList()
print(spin_obtlist)
print('..finished generating determinants in %.3f seconds.\n' % (time.time() - t))

print('Generating Hamiltonian Matrix...')
t = time.time()
Hamiltonian_generator = HamiltonianGenerator(H, MO)
Hamiltonian_matrix = Hamiltonian_generator.generateMatrix(detList)
print(Hamiltonian_matrix)

print('..finished generating Matrix in %.3f seconds.\n' % (time.time() - t))

print('Diagonalizing Hamiltonian Matrix...')


Generating 3 CIS singlet Determinants...
[0, 1]
..finished generating determinants in 0.001 seconds.

Generating Hamiltonian Matrix...
[[-1.07826  0.19679  0.     ]
 [ 0.19679 -1.07826  0.     ]
 [ 0.       0.      -1.59529]]
..finished generating Matrix in 0.001 seconds.

Diagonalizing Hamiltonian Matrix...


In [10]:
numDet = len(detList)
print(numDet)
for i in range(numDet):
    for j in range(numDet):
        unique1, unique2, sign = detList[i].getUniqueOrbitalsInMixIndexListsPlusSign(detList[j])
        print(detList[i], detList[j], unique1, unique2, sign)

3
|[1][0]> |[1][0]> [] [] 1
|[1][0]> |[0][1]> [2, 1] [0, 3] 1
|[1][0]> |[0][0]> [2] [0] 1
|[0][1]> |[1][0]> [0, 3] [2, 1] 1
|[0][1]> |[0][1]> [] [] 1
|[0][1]> |[0][0]> [3] [1] 1
|[0][0]> |[1][0]> [0] [2] 1
|[0][0]> |[0][1]> [1] [3] 1
|[0][0]> |[0][0]> [] [] 1


In [6]:
H_elm = 0
for m in spin_obtlist:
    H_elm += H[m,m]
    print(m,m,H_elm)
length = len(spin_obtlist)
for m in range(length-1):
    for n in range(m+1, length):
        H_elm += MO[m, n, m, n ]
        print(m,n,m,n,H_elm)
        
print(H_elm)

0 0 -1.110844179650526
1 1 -2.221688359301052
0 1 0 1 -1.595285859855316
-1.595285859855316


In [ ]:
unique1, unique2, sign = det1.getUniqueOrbitalsInMixIndexListsPlusSign(det2)
        return sign * self.antiSym2eInt[unique1[0], unique1[1], unique2[0], unique2[1]]

In [ ]:
t = time.time()

e_cis, wavefunctions = np.linalg.eigh(Hamiltonian_matrix)
print('..finished diagonalization in %.3f seconds.\n' % (time.time() - t))

print('# Determinants:     % 16d' % (len(detList)))

print('SCF energy:         % 16.10f' % (scf_e))

hartree2eV = 27.211

print('\nCIS Excitation Energies (Singlets only):')
print(' #        Hartree                  eV')
print('--  --------------------  --------------------')
for i in range(1, len(e_cis)):
    excit_e = e_cis[i] + mol.nuclear_repulsion_energy() - scf_e
    print('%2d %20.10f %20.10f' % (i, excit_e, excit_e * hartree2eV))